<a href="https://colab.research.google.com/github/swgu931/python_wrangling_munging/blob/main/kosdaq_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

❌ 프로그램 실행 중 오류 발생: 토큰 발급 실패: {"error_description":"유효하지 않은 AppKey입니다.","error_code":"EGW00103"}


In [ ]:
# prompt: 코스닥 주식 매매량을 볼 수 있는 프로그램을 작성해줘 입력은 코스닥 등록된 회사명이 리스트로 들어가고 출력은 1. 각 회사의 총 시장에 풀린 주식수와 현재 주가,
# 2. 최근 일주일 기관별 매수 수량. 매도수량. 가격 , 프로그램 매매의 매수 수량. 매도수량. 가격. 3. 외국인 매수수량. 매도수량. 가격,  4.  결과를 csv 파일에 저장해주고, 파일명은 "오늘 날짜- stockanalysis.csv" 로 해죠. (예: 20250531-stockanalysis.csv

!pip install yfinance pandas openpyxl

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time

def get_stock_data(stock_list):
    """
    Fetches stock data for a list of KOSDAQ companies.

    Args:
        stock_list (list): A list of KOSDAQ company names.

    Returns:
        tuple: A tuple containing three dictionaries:
            - total_shares_price: Total outstanding shares and current price per company.
            - institutional_trading: Institutional trading volume and price per company.
            - foreign_trading: Foreign trading volume and price per company.
    """
    total_shares_price = {}
    institutional_trading = {}
    foreign_trading = {}
    end_date = datetime.now()
    start_date = end_date - timedelta(days=7)

    for company in stock_list:
        try:
            # Attempt to find the ticker symbol for the company.
            # This requires a mapping or a search mechanism.
            # For simplicity, this example assumes the input is the ticker symbol.
            # In a real-world scenario, you would need a way to map company names to tickers.
            ticker = yf.Ticker(company + ".KQ") # Assuming .KQ for KOSDAQ

            # 1. Total outstanding shares and current price
            info = ticker.info
            total_shares = info.get('sharesOutstanding', 'N/A')
            current_price = info.get('currentPrice', 'N/A')
            total_shares_price[company] = {'Total Shares Outstanding': total_shares, 'Current Price': current_price}

            # Get historical data for the last week
            hist_data = ticker.history(start=start_date, end=end_date)

            if not hist_data.empty:
                # We can't directly get institutional, foreign, and program trading data
                # from yfinance's standard historical data. This data is typically available
                # from specialized financial data providers.

                # For demonstration purposes, we'll simulate this data or leave it as 'N/A'.
                # In a real application, you would integrate with a financial data API
                # that provides this breakdown (e.g., FnGuide, DataGuide).

                # 2. Recent weekly institutional and program trading (simulated)
                institutional_buy_volume = 'N/A' # Replace with actual data fetching
                institutional_sell_volume = 'N/A' # Replace with actual data fetching
                institutional_avg_price = 'N/A' # Replace with actual data fetching

                program_buy_volume = 'N/A' # Replace with actual data fetching
                program_sell_volume = 'N/A' # Replace with actual data fetching
                program_avg_price = 'N/A' # Replace with actual data fetching

                institutional_trading[company] = {
                    'Institutional Buy Volume (Weekly)': institutional_buy_volume,
                    'Institutional Sell Volume (Weekly)': institutional_sell_volume,
                    'Institutional Average Price (Weekly)': institutional_avg_price,
                    'Program Buy Volume (Weekly)': program_buy_volume,
                    'Program Sell Volume (Weekly)': program_sell_volume,
                    'Program Average Price (Weekly)': program_avg_price
                }

                # 3. Recent weekly foreign trading (simulated)
                foreign_buy_volume = 'N/A' # Replace with actual data fetching
                foreign_sell_volume = 'N/A' # Replace with actual data fetching
                foreign_avg_price = 'N/A' # Replace with actual data fetching

                foreign_trading[company] = {
                    'Foreign Buy Volume (Weekly)': foreign_buy_volume,
                    'Foreign Sell Volume (Weekly)': foreign_sell_volume,
                    'Foreign Average Price (Weekly)': foreign_avg_price
                }

            else:
                 institutional_trading[company] = {
                    'Institutional Buy Volume (Weekly)': 'No data',
                    'Institutional Sell Volume (Weekly)': 'No data',
                    'Institutional Average Price (Weekly)': 'No data',
                    'Program Buy Volume (Weekly)': 'No data',
                    'Program Sell Volume (Weekly)': 'No data',
                    'Program Average Price (Weekly)': 'No data'
                }
                 foreign_trading[company] = {
                    'Foreign Buy Volume (Weekly)': 'No data',
                    'Foreign Sell Volume (Weekly)': 'No data',
                    'Foreign Average Price (Weekly)': 'No data'
                }


        except Exception as e:
            print(f"Could not retrieve data for {company}: {e}")
            total_shares_price[company] = {'Total Shares Outstanding': 'Error', 'Current Price': 'Error'}
            institutional_trading[company] = {
                'Institutional Buy Volume (Weekly)': 'Error',
                'Institutional Sell Volume (Weekly)': 'Error',
                'Institutional Average Price (Weekly)': 'Error',
                'Program Buy Volume (Weekly)': 'Error',
                'Program Sell Volume (Weekly)': 'Error',
                'Program Average Price (Weekly)': 'Error'
            }
            foreign_trading[company] = {
                'Foreign Buy Volume (Weekly)': 'Error',
                'Foreign Sell Volume (Weekly)': 'Error',
                'Foreign Average Price (Weekly)': 'Error'
            }
        # Add a small delay to avoid hitting API limits
        time.sleep(1)

    return total_shares_price, institutional_trading, foreign_trading

def save_to_csv(total_shares_price, institutional_trading, foreign_trading):
    """
    Saves the fetched stock data to a CSV file.

    Args:
        total_shares_price (dict): Total outstanding shares and current price data.
        institutional_trading (dict): Institutional trading data.
        foreign_trading (dict): Foreign trading data.
    """
    all_data = []
    for company in total_shares_price:
        row = {'Company': company}
        row.update(total_shares_price.get(company, {}))
        row.update(institutional_trading.get(company, {}))
        row.update(foreign_trading.get(company, {}))
        all_data.append(row)

    df = pd.DataFrame(all_data)

    today = datetime.now().strftime('%Y%m%d')
    filename = f"{today}-stockanalysis.csv"

    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

# Example usage:
# Replace with the actual KOSDAQ company names or ticker symbols you want to analyze

kosdaq_companies = ["035720", "005930"] # Example: Kakao (035720.KQ), Samsung Electronics (005930.KS) - Note: Samsung is KOSPI, adjust if needed.
# You need to provide valid KOSDAQ ticker symbols.

# total_shares_price_data, institutional_trading_data, foreign_trading_data = get_stock_data(kosdaq_companies)
# save_to_csv(total_shares_price_data, institutional_trading_data, foreign_trading_data)

In [ ]:
kosdaq_companies = ["034020", "042660"] # Example: Kakao (035720.KQ), Samsung Electronics (005930.KS) - Note: Samsung is KOSPI, adjust if needed.
# You need to provide valid KOSDAQ ticker symbols.

total_shares_price_data, institutional_trading_data, foreign_trading_data = get_stock_data(kosdaq_companies)
save_to_csv(total_shares_price_data, institutional_trading_data, foreign_trading_data)

Data saved to 20250531-stockanalysis.csv


In [ ]:
!ls

20250531-stockanalysis.csv  sample_data


In [ ]:
# prompt: 공공 데이터 포털을 이용해서 다시 코드를 수정해줘

import requests
import pandas as pd
from datetime import datetime, timedelta
from io import BytesIO
import xml.etree.ElementTree as ET

# Public Data Portal Service Key
# Replace with your actual Service Key
# Get your key from https://www.data.go.kr/
service_key = "YOUR_SERVICE_KEY"

def get_stock_data_from_data_go_kr(stock_list):
    """
    Fetches stock data for a list of KOSDAQ companies from the Public Data Portal.

    Args:
        stock_list (list): A list of KOSDAQ stock codes.

    Returns:
        tuple: A tuple containing two dictionaries:
            - total_shares_price: Total outstanding shares and current price per company (limited data from this API).
            - trading_data: Institutional, Foreign, and Program trading data per company.
    """
    total_shares_price = {}
    trading_data = {}

    # API Endpoint for Stock Trading Trends by Investor Type
    url = "http://apis.data.go.kr/1161000/insttTrderSntmtInfoService/getInsttTrderSntmtInfo"

    for stock_code in stock_list:
        try:
            # Get today's date and yesterday's date for query
            today = datetime.now().strftime('%Y%m%d')
            yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')

            params = {
                'serviceKey': service_key,
                'numOfRows': '10',
                'pageNo': '1',
                'resultType': 'XML', # Can also use JSON
                'itmsNm': '', # Item Name (can be empty if you use isinCd)
                'mrktCtg': 'KOSDAQ', # Market Category (KOSPI, KOSDAQ)
                'clsgn': stock_code, # Stock Code
                'trdDd': yesterday # Trading Date (data is typically for the previous trading day)
            }

            response = requests.get(url, params=params)
            response.raise_for_status() # Raise an exception for bad status codes

            # Process the XML response
            root = ET.fromstring(response.content)
            items = root.findall('.//item')

            if items:
                # We'll take the first item as it's for the date we queried
                item = items[0]
                company_name = item.find('itmsNm').text if item.find('itmsNm') is not None else 'N/A'
                stock_code_returned = item.find('isinCd').text if item.find('isinCd') is not None else 'N/A'
                trd_dd = item.find('trdDd').text if item.find('trdDd') is not None else 'N/A'

                # Data available from this API call (focused on investor trading)
                inst_net_buy_vol = item.find('insttNetbuyVol').text if item.find('insttNetbuyVol') is not None else 'N/A'
                frgrn_net_buy_vol = item.find('frgrnNetbuyVol').text if item.find('frgrnNetbuyVol') is not None else 'N/A'
                prgm_net_buy_vol = item.find('prgmNetbuyVol').text if item.find('prgmNetbuyVol') is not None else 'N/A'

                # This API does not provide total outstanding shares or current price directly in this specific call.
                # You would need to use a different API endpoint or source for that information.
                total_shares_price[stock_code] = {'Company Name': company_name, 'Total Shares Outstanding': 'N/A (DataGoKr)', 'Current Price': 'N/A (DataGoKr)'}

                trading_data[stock_code] = {
                    'Company Name': company_name,
                    'Trading Date': trd_dd,
                    'Institutional Net Buy Volume (Yesterday)': inst_net_buy_vol,
                    'Foreign Net Buy Volume (Yesterday)': frgrn_net_buy_vol,
                    'Program Net Buy Volume (Yesterday)': prgm_net_buy_vol
                    # Note: This API provides Net Buy Volume, not separate Buy/Sell volumes.
                    # It also doesn't provide average price.
                }
            else:
                print(f"No data found for stock code {stock_code} on {yesterday}")
                total_shares_price[stock_code] = {'Company Name': 'N/A', 'Total Shares Outstanding': 'No data', 'Current Price': 'No data'}
                trading_data[stock_code] = {
                    'Company Name': 'N/A',
                    'Trading Date': 'No data',
                    'Institutional Net Buy Volume (Yesterday)': 'No data',
                    'Foreign Net Buy Volume (Yesterday)': 'No data',
                    'Program Net Buy Volume (Yesterday)': 'No data'
                }

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {stock_code}: {e}")
            total_shares_price[stock_code] = {'Company Name': 'Error', 'Total Shares Outstanding': 'Error', 'Current Price': 'Error'}
            trading_data[stock_code] = {
                'Company Name': 'Error',
                'Trading Date': 'Error',
                'Institutional Net Buy Volume (Yesterday)': 'Error',
                'Foreign Net Buy Volume (Yesterday)': 'Error',
                'Program Net Buy Volume (Yesterday)': 'Error'
            }
        except ET.ParseError as e:
             print(f"Error parsing XML for {stock_code}: {e}")
             total_shares_price[stock_code] = {'Company Name': 'Parse Error', 'Total Shares Outstanding': 'Parse Error', 'Current Price': 'Parse Error'}
             trading_data[stock_code] = {
                'Company Name': 'Parse Error',
                'Trading Date': 'Parse Error',
                'Institutional Net Buy Volume (Yesterday)': 'Parse Error',
                'Foreign Net Buy Volume (Yesterday)': 'Parse Error',
                'Program Net Buy Volume (Yesterday)': 'Parse Error'
            }

    return total_shares_price, trading_data

def save_data_go_kr_to_csv(total_shares_price, trading_data):
    """
    Saves the fetched stock data from Data Go Kr to a CSV file.

    Args:
        total_shares_price (dict): Total outstanding shares and current price data (limited).
        trading_data (dict): Institutional, Foreign, and Program trading data.
    """
    all_data = []
    # Combine data from both dictionaries, using stock code as the key
    for stock_code in set(list(total_shares_price.keys()) + list(trading_data.keys())):
        row = {'Stock Code': stock_code}
        row.update(total_shares_price.get(stock_code, {}))
        row.update(trading_data.get(stock_code, {}))
        all_data.append(row)

    df = pd.DataFrame(all_data)

    today = datetime.now().strftime('%Y%m%d')
    filename = f"{today}-stockanalysis_datagokr.csv"

    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

# Example usage:
# Replace with the actual KOSDAQ stock codes you want to analyze
# Find stock codes on financial websites or the Korea Exchange (KRX) website.
kosdaq_codes = ["034020", "042660"] # Example KOSDAQ codes

# --- Important ---
# You MUST replace "YOUR_SERVICE_KEY" with your actual Service Key obtained from data.go.kr.
# This code will NOT work without a valid Service Key.
# -----------------

# total_shares_price_data_dgk, trading_data_dgk = get_stock_data_from_data_go_kr(kosdaq_codes)
# save_data_go_kr_to_csv(total_shares_price_data_dgk, trading_data_dgk)



In [ ]:
kosdaq_codes = ["034020", "042660"] # Example KOSDAQ codes
total_shares_price_data_dgk, trading_data_dgk = get_stock_data_from_data_go_kr(kosdaq_codes)
save_data_go_kr_to_csv(total_shares_price_data_dgk, trading_data_dgk)

NameError: name 'get_stock_data_from_data_go_kr' is not defined

In [ ]:
# prompt: 위 코드를 오류를 수정해서 수정해줘

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
import requests
from io import BytesIO
import xml.etree.ElementTree as ET

# Install necessary libraries
!pip install yfinance pandas openpyxl

def get_stock_data(stock_list):
    """
    Fetches limited stock data for a list of KOSDAQ companies using yfinance.
    Note: yfinance does not directly provide institutional/foreign/program trading data
    for Korean markets. This function primarily provides total shares and current price.

    Args:
        stock_list (list): A list of KOSDAQ stock codes.

    Returns:
        tuple: A tuple containing three dictionaries:
            - total_shares_price: Total outstanding shares and current price per company.
            - institutional_trading: Placeholder data ('N/A' or 'No data').
            - foreign_trading: Placeholder data ('N/A' or 'No data').
    """
    total_shares_price = {}
    institutional_trading = {}
    foreign_trading = {}
    end_date = datetime.now()
    # Fetch data for the last 7 days, including today
    start_date = end_date - timedelta(days=7)

    for stock_code in stock_list:
        try:
            # Append .KQ for KOSDAQ tickers in yfinance
            ticker = yf.Ticker(stock_code + ".KQ")

            # 1. Total outstanding shares and current price
            info = ticker.info
            total_shares = info.get('sharesOutstanding', 'N/A')
            current_price = info.get('currentPrice', 'N/A')
            total_shares_price[stock_code] = {'Total Shares Outstanding': total_shares, 'Current Price': current_price}

            # Get historical data for the last week
            hist_data = ticker.history(start=start_date, end=end_date)

            if not hist_data.empty:
                # yfinance does NOT provide institutional, foreign, and program trading data
                # breakdown for individual stock history.
                # The following lines remain as placeholders indicating this data is missing.
                institutional_trading[stock_code] = {
                    'Institutional Buy Volume (Weekly)': 'N/A (yfinance)',
                    'Institutional Sell Volume (Weekly)': 'N/A (yfinance)',
                    'Institutional Average Price (Weekly)': 'N/A (yfinance)',
                    'Program Buy Volume (Weekly)': 'N/A (yfinance)',
                    'Program Sell Volume (Weekly)': 'N/A (yfinance)',
                    'Program Average Price (Weekly)': 'N/A (yfinance)'
                }

                foreign_trading[stock_code] = {
                    'Foreign Buy Volume (Weekly)': 'N/A (yfinance)',
                    'Foreign Sell Volume (Weekly)': 'N/A (yfinance)',
                    'Foreign Average Price (Weekly)': 'N/A (yfinance)'
                }

            else:
                 print(f"No historical data found for {stock_code} in the last week.")
                 institutional_trading[stock_code] = {
                    'Institutional Buy Volume (Weekly)': 'No data',
                    'Institutional Sell Volume (Weekly)': 'No data',
                    'Institutional Average Price (Weekly)': 'No data',
                    'Program Buy Volume (Weekly)': 'No data',
                    'Program Sell Volume (Weekly)': 'No data',
                    'Program Average Price (Weekly)': 'No data'
                }
                 foreign_trading[stock_code] = {
                    'Foreign Buy Volume (Weekly)': 'No data',
                    'Foreign Sell Volume (Weekly)': 'No data',
                    'Foreign Average Price (Weekly)': 'No data'
                }


        except Exception as e:
            print(f"Could not retrieve yfinance data for {stock_code}: {e}")
            total_shares_price[stock_code] = {'Total Shares Outstanding': 'Error', 'Current Price': 'Error'}
            institutional_trading[stock_code] = {
                'Institutional Buy Volume (Weekly)': 'Error',
                'Institutional Sell Volume (Weekly)': 'Error',
                'Institutional Average Price (Weekly)': 'Error',
                'Program Buy Volume (Weekly)': 'Error',
                'Program Sell Volume (Weekly)': 'Error',
                'Program Average Price (Weekly)': 'Error'
            }
            foreign_trading[stock_code] = {
                'Foreign Buy Volume (Weekly)': 'Error',
                'Foreign Sell Volume (Weekly)': 'Error',
                'Foreign Average Price (Weekly)': 'Error'
            }
        # Add a small delay to avoid hitting API limits
        time.sleep(1)

    return total_shares_price, institutional_trading, foreign_trading

def save_to_csv(total_shares_price, institutional_trading, foreign_trading):
    """
    Saves the fetched stock data from yfinance to a CSV file.

    Args:
        total_shares_price (dict): Total outstanding shares and current price data.
        institutional_trading (dict): Institutional trading data (placeholder).
        foreign_trading (dict): Foreign trading data (placeholder).
    """
    all_data = []
    # Combine data from all three dictionaries, using stock code as the key
    all_keys = set(list(total_shares_price.keys()) + list(institutional_trading.keys()) + list(foreign_trading.keys()))
    for stock_code in all_keys:
        row = {'Stock Code': stock_code}
        row.update(total_shares_price.get(stock_code, {}))
        row.update(institutional_trading.get(stock_code, {}))
        row.update(foreign_trading.get(stock_code, {}))
        all_data.append(row)


    df = pd.DataFrame(all_data)

    today = datetime.now().strftime('%Y%m%d')
    filename = f"{today}-stockanalysis_yfinance.csv"

    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"yfinance data saved to {filename}")

# Public Data Portal Service Key
# Replace with your actual Service Key
# Get your key from https://www.data.go.kr/
service_key = "YOUR_SERVICE_KEY" # <-- MAKE SURE TO REPLACE THIS

def get_stock_data_from_data_go_kr(stock_list):
    """
    Fetches stock trading data by investor type for a list of KOSDAQ companies
    from the Public Data Portal.

    Args:
        stock_list (list): A list of KOSDAQ stock codes.

    Returns:
        tuple: A tuple containing two dictionaries:
            - total_shares_price: Placeholder data ('N/A') as this API doesn't provide it directly.
            - trading_data: Institutional, Foreign, and Program net trading volume per company.
    """
    total_shares_price = {} # Placeholder, as this API doesn't provide this data
    trading_data = {}

    # API Endpoint for Stock Trading Trends by Investor Type
    url = "http://apis.data.go.kr/1161000/insttTrderSntmtInfoService/getInsttTrderSntmtInfo"

    for stock_code in stock_list:
        try:
            # Get today's date and yesterday's date for query
            # Data is typically for the previous trading day
            yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')

            params = {
                'serviceKey': service_key,
                'numOfRows': '10', # You might adjust this if you need more historical days
                'pageNo': '1',
                'resultType': 'XML', # Can also use JSON, but XML is used in original code
                'itmsNm': '', # Item Name (can be empty if you use isinCd)
                'mrktCtg': 'KOSDAQ', # Market Category (KOSPI, KOSDAQ)
                'isinCd': stock_code, # Use isinCd (stock code) for specific lookup
                'trdDd': yesterday # Trading Date
            }

            response = requests.get(url, params=params)
            response.raise_for_status() # Raise an exception for bad status codes

            # Process the XML response
            # Use BytesIO to handle the response content as a file-like object
            root = ET.fromstring(response.content)
            items = root.findall('.//item')

            if items:
                # We'll take the first item as it's for the date we queried
                item = items[0]
                company_name = item.find('itmsNm').text if item.find('itmsNm') is not None else 'N/A'
                stock_code_returned = item.find('isinCd').text if item.find('isinCd') is not None else 'N/A'
                trd_dd = item.find('trdDd').text if item.find('trdDd') is not None else 'N/A'

                # Data available from this API call (focused on investor trading)
                # These are Net Buy Volumes
                inst_net_buy_vol = item.find('insttNetbuyVol').text if item.find('insttNetbuyVol') is not None else 'N/A'
                frgrn_net_buy_vol = item.find('frgrnNetbuyVol').text if item.find('frgrnNetbuyVol') is not None else 'N/A'
                prgm_net_buy_vol = item.find('prgmNetbuyVol').text if item.find('prgmNetbuyVol') is not None else 'N/A'

                # This API does not provide total outstanding shares or current price directly in this specific call.
                # Keeping this as a placeholder
                total_shares_price[stock_code] = {'Company Name (DataGoKr)': company_name, 'Total Shares Outstanding': 'N/A (DataGoKr)', 'Current Price': 'N/A (DataGoKr)'}

                trading_data[stock_code] = {
                    'Company Name (DataGoKr)': company_name,
                    'Trading Date': trd_dd,
                    'Institutional Net Buy Volume (Yesterday)': inst_net_buy_vol,
                    'Foreign Net Buy Volume (Yesterday)': frgrn_net_buy_vol,
                    'Program Net Buy Volume (Yesterday)': prgm_net_buy_vol
                    # Note: This API provides Net Buy Volume, not separate Buy/Sell volumes.
                    # It also doesn't provide average price.
                }
            else:
                print(f"No data found for stock code {stock_code} on {yesterday} from DataGoKr")
                total_shares_price[stock_code] = {'Company Name (DataGoKr)': 'N/A', 'Total Shares Outstanding': 'No data', 'Current Price': 'No data'}
                trading_data[stock_code] = {
                    'Company Name (DataGoKr)': 'N/A',
                    'Trading Date': 'No data',
                    'Institutional Net Buy Volume (Yesterday)': 'No data',
                    'Foreign Net Buy Volume (Yesterday)': 'No data',
                    'Program Net Buy Volume (Yesterday)': 'No data'
                }

        except requests.exceptions.RequestException as e:
            print(f"Error fetching DataGoKr data for {stock_code}: {e}")
            total_shares_price[stock_code] = {'Company Name (DataGoKr)': 'Error', 'Total Shares Outstanding': 'Error', 'Current Price': 'Error'}
            trading_data[stock_code] = {
                'Company Name (DataGoKr)': 'Error',
                'Trading Date': 'Error',
                'Institutional Net Buy Volume (Yesterday)': 'Error',
                'Foreign Net Buy Volume (Yesterday)': 'Error',
                'Program Net Buy Volume (Yesterday)': 'Error'
            }
        except ET.ParseError as e:
             print(f"Error parsing XML for {stock_code} from DataGoKr: {e}")
             total_shares_price[stock_code] = {'Company Name (DataGoKr)': 'Parse Error', 'Total Shares Outstanding': 'Parse Error', 'Current Price': 'Parse Error'}
             trading_data[stock_code] = {
                'Company Name (DataGoKr)': 'Parse Error',
                'Trading Date': 'Parse Error',
                'Institutional Net Buy Volume (Yesterday)': 'Parse Error',
                'Foreign Net Buy Volume (Yesterday)': 'Parse Error',
                'Program Net Buy Volume (Yesterday)': 'Parse Error'
            }
        # Add a small delay
        time.sleep(1)


    return total_shares_price, trading_data

def save_data_go_kr_to_csv(total_shares_price, trading_data):
    """
    Saves the fetched stock data from Data Go Kr to a CSV file.

    Args:
        total_shares_price (dict): Total outstanding shares and current price data (limited/placeholder).
        trading_data (dict): Institutional, Foreign, and Program net trading data.
    """
    all_data = []
    # Combine data from both dictionaries, using stock code as the key
    all_keys = set(list(total_shares_price.keys()) + list(trading_data.keys()))
    for stock_code in all_keys:
        row = {'Stock Code': stock_code}
        row.update(total_shares_price.get(stock_code, {}))
        row.update(trading_data.get(stock_code, {}))
        all_data.append(row)

    df = pd.DataFrame(all_data)

    today = datetime.now().strftime('%Y%m%d')
    filename = f"{today}-stockanalysis_datagokr.csv"

    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"DataGoKr data saved to {filename}")

# --- Example Usage ---
# Replace with the actual KOSDAQ stock codes you want to analyze
# Find stock codes on financial websites or the Korea Exchange (KRX) website.

# Using yfinance (primarily for total shares and current price, limited history)
kosdaq_codes_yfinance = ["034020", "042660"] # Example KOSDAQ codes
print("\n--- Fetching data using yfinance ---")
total_shares_price_data_yf, institutional_trading_data_yf, foreign_trading_data_yf = get_stock_data(kosdaq_codes_yfinance)
save_to_csv(total_shares_price_data_yf, institutional_trading_data_yf, foreign_trading_data_yf)
!ls

# Using Data Go Kr (primarily for institutional/foreign/program net trading volumes for the previous day)
kosdaq_codes_datagokr = ["034020", "042660"] # Example KOSDAQ codes
print("\n--- Fetching data using Data Go Kr ---")
# --- Important ---
# You MUST replace "YOUR_SERVICE_KEY" with your actual Service Key obtained from data.go.kr.
# This code will NOT work without a valid Service Key.
# -----------------
if service_key == "YOUR_SERVICE_KEY":
    print("Please replace 'YOUR_SERVICE_KEY' with your actual Service Key from data.go.kr to fetch data from the Public Data Portal.")
else:
    total_shares_price_data_dgk, trading_data_dgk = get_stock_data_from_data_go_kr(kosdaq_codes_datagokr)
    save_data_go_kr_to_csv(total_shares_price_data_dgk, trading_data_dgk)
    !ls



--- Fetching data using yfinance ---
yfinance data saved to 20250531-stockanalysis_yfinance.csv
20250531-stockanalysis.csv  20250531-stockanalysis_yfinance.csv  sample_data

--- Fetching data using Data Go Kr ---
Please replace 'YOUR_SERVICE_KEY' with your actual Service Key from data.go.kr to fetch data from the Public Data Portal.


In [ ]:
# prompt: 위 코드를  data.krx.co.kr  에 접속해서 분석하는 코드로 수정해줘

# This code focuses on accessing data from data.krx.co.kr or related public portals.
# Note: Direct web scraping of data.krx.co.kr might be subject to terms of use
# and robots.txt restrictions. The previous code used the Public Data Portal (data.go.kr) API,
# which is a more official and stable way to access government/public data, including KRX data.

# To access data from data.krx.co.kr programmatically, you generally need to:
# 1. Identify the specific data you want (e.g., daily stock prices, trading volumes,
#    investor trends).
# 2. Find if KRX provides an official API for this data. The Public Data Portal
#    often acts as a gateway to KRX data via specific APIs.
# 3. If there's no official API for the exact data you need and it's available on the
#    website, you might need to explore web scraping techniques. However, this is
#    less reliable and potentially against the site's terms.

# The previous code already implemented fetching investor trading data using the
# Public Data Portal API. This is the recommended approach.

# If you specifically need to access other data endpoints or navigate the KRX website
# structure, you would typically use the `requests` library to make HTTP requests
# and potentially libraries like `BeautifulSoup` or `lxml` to parse HTML content.

# However, without a specific task (e.g., "fetch daily closing prices from KRX's daily
# price page"), providing a general KRX access code is difficult.

# Let's assume you want to adapt the previous code to *verify* or *supplement* the
# data from data.go.kr by trying to access publicly available data on data.krx.co.kr
# using HTTP requests.
# This is complex and might not work reliably due to website structure changes,
# JavaScript loading, or anti-scraping measures.

# Example: Trying to access a hypothetical daily price page (this URL is illustrative
# and likely not the actual direct data endpoint).
# A more realistic approach involves finding specific AJAX endpoints or API calls
# the website uses.

import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# The direct data pages on data.krx.co.kr often require specific parameters,
# like dates, markets, and sometimes require sending POST requests or
# navigating through JavaScript-heavy interfaces.

# A common pattern for programmatic access is to find the endpoint that serves
# the data in a machine-readable format (like CSV, JSON, or XML) or the endpoint
# that powers charts/tables on the website. This usually involves inspecting
# network requests in a web browser's developer tools.

# Let's create a function that *attempts* to fetch data from a hypothetical
# endpoint for daily prices. This is highly dependent on the actual KRX website
# structure and might need significant adjustments based on what data you target.

def get_krx_daily_prices(stock_code, start_date_str, end_date_str):
    """
    Attempts to fetch daily price data for a stock from a hypothetical KRX endpoint.
    This is highly illustrative and may not work due to the complexity of the
    KRX website's data delivery.

    Args:
        stock_code (str): The stock code (e.g., "034020").
        start_date_str (str): Start date in 'YYYYMMDD' format.
        end_date_str (str): End date in 'YYYYMMDD' format.

    Returns:
        pandas.DataFrame or None: DataFrame with price data, or None if fetching fails.
    """
    # Disclaimer: This URL and parameters are illustrative. You need to find the
    # actual endpoint by inspecting KRX website's network traffic.
    # KRX's data access often involves POST requests and complex parameters.
    # A simple GET request to a predictable URL is often not sufficient.

    # A more common way to get data from KRX interfaces is via their data download center
    # or finding the AJAX requests that load data tables.

    # Example of a potential API-like endpoint often used by websites (highly speculative):
    # You would find this by using developer tools in your browser on data.krx.co.kr
    # and observing the requests made when you load data.
    api_url = "http://data.krx.co.kr/comm/bldAttendant/getBldAttendant.cmd" # This is a known endpoint for some KRX data queries

    # Parameters for a daily price query are complex and require finding the correct `bld` (build) code
    # and other form data. You'd typically find this using browser developer tools.
    # For example, fetching daily stock prices might involve 'bld=dbms/MDC/STAT/STD/MDCSTAT01501'.
    # Let's construct a placeholder request with some common parameters.

    headers = {
        'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020501', # Referer might be needed
        'User-Agent': 'Mozilla/5.0' # Mimic a browser
    }

    params = {
        'bld': 'dbms/MDC/STAT/STD/MDCSTAT01501', # Example bld code for daily price statistics
        'locale': 'ko_KR',
        'tboxisuCd_finder_stkisu0_0': stock_code, # Stock code input
        'isuCd': stock_code, # ISU code
        'isuNm': '종목명', # Placeholder - finding the correct name might be needed
        'dataTp': 'A', # Data type? (Speculative)
        'shareTp': '1', # Share type? (Speculative)
        'stdDd': end_date_str, # Standard date (often the end date for a range query)
        'startDate': start_date_str,
        'endDate': end_date_str,
        'trdDd': '20231026', # Trading date for single-day queries (if applicable)
        'money': '1', # Display money in billions? (Speculative)
        'csvTp': 'csv', # Request CSV format? (Speculative - often requires a separate download endpoint)
        'name': 'fileDown', # Parameter often used to trigger a download
        'file_name': '주식시세일자별', # Suggested file name
        'url': 'dbms/MDC/STAT/STD/MDCSTAT01501' # URL parameter
    }

    # Note: Directly getting CSV via this endpoint and parameters might not work.
    # KRX often has a separate download endpoint that receives parameters from a
    # previous query.

    # Let's attempt a POST request which is more common for KRX data queries.
    # The parameters for POST requests are often sent in the request body as form data.
    # Finding the exact payload requires inspecting the browser's network requests.

    post_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generateOTP.do"

    # The payload for generating an OTP (One-Time Password) needed for downloading
    # data after making a query. This payload is complex and depends on the specific query.
    # Example payload structure (highly dependent on the specific report/data):
    otp_payload = {
        'bld': 'dbms/MDC/STAT/STD/MDCSTAT01501',
        'locale': 'ko_KR',
        'tboxisuCd_finder_stkisu0_0': stock_code,
        'isuCd': stock_code,
        'isuNm': '종목명', # Needs correct name
        'dataTp': 'A',
        'shareTp': '1',
        'stdDd': end_date_str,
        'startDate': start_date_str,
        'endDate': end_date_str,
        'trdDd': end_date_str, # Use end date for range queries
        'money': '1',
        'csvTp': 'csv',
        'name': 'fileDown',
        'file_name': '주식시세일자별',
        'url': 'dbms/MDC/STAT/STD/MDCSTAT01501'
    }

    try:
        # Step 1: Generate OTP
        otp_response = requests.post(post_url, otp_payload, headers=headers)
        otp_response.raise_for_status()
        otp = otp_response.text # The response should be the OTP string

        if not otp:
            print(f"Failed to generate OTP for {stock_code}. Check parameters.")
            return None

        # Step 2: Download the file using the OTP
        download_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.do"
        download_payload = {
            'code': otp # Use the obtained OTP
        }

        download_response = requests.post(download_url, download_payload, headers=headers)
        download_response.raise_for_status()

        # Read the content into a pandas DataFrame
        # The content is typically bytes, use BytesIO
        df = pd.read_csv(BytesIO(download_response.content), encoding='euc-kr') # KRX often uses euc-kr encoding

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error accessing KRX data for {stock_code}: {e}")
        print("This might be due to incorrect endpoints, parameters, or website changes.")
        print("Consider using the Public Data Portal API (data.go.kr) as demonstrated previously,")
        print("which is a more stable source for KRX data.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred for {stock_code}: {e}")
        return None


# Example Usage:
# --- IMPORTANT ---
# The `get_krx_daily_prices` function above is highly experimental and
# relies on finding the correct, potentially unstable, internal API endpoints
# and parameters used by data.krx.co.kr.
# It is strongly recommended to use the Public Data Portal API (data.go.kr)
# if it provides the necessary data, as it is an official and stable source.
# You MUST replace "YOUR_SERVICE_KEY" with your actual Service Key for the data.go.kr part.
# -----------------

kosdaq_codes_krx_attempt = ["034020", "042660"] # Example KOSDAQ codes
end_date_str = datetime.now().strftime('%Y%m%d')
start_date_str = (datetime.now() - timedelta(days=7)).strftime('%Y%m%d')

print("\n--- Attempting to fetch daily prices directly from data.krx.co.kr (Experimental) ---")

# results_krx_attempt = {}
# for code in kosdaq_codes_krx_attempt:
#     print(f"Fetching daily prices for {code} from KRX...")
#     df_krx = get_krx_daily_prices(code, start_date_str, end_date_str)
#     if df_krx is not None:
#         print(f"Successfully fetched data for {code}. Head:")
#         print(df_krx.head())
#         results_krx_attempt[code] = df_krx
#     else:
#         print(f"Failed to fetch data for {code} from KRX.")
#     time.sleep(2) # Be polite when attempting to access websites

# print("\n--- End of KRX direct access attempt ---")


# --- Re-running the Data Go Kr example from the previous code for comparison ---
# This is the RECOMMENDED approach for stable data access.

kosdaq_codes_datagokr = ["034020", "042660"] # Example KOSDAQ codes
print("\n--- Fetching data using Data Go Kr (Recommended) ---")

# --- Important ---
# You MUST replace "YOUR_SERVICE_KEY" with your actual Service Key obtained from data.go.kr.
# This code will NOT work without a valid Service Key.
# -----------------
service_key = "YOUR_SERVICE_KEY" # <-- MAKE SURE TO REPLACE THIS

if service_key == "YOUR_SERVICE_KEY":
    print("Please replace 'YOUR_SERVICE_KEY' with your actual Service Key from data.go.kr to fetch data from the Public Data Portal.")
else:
    total_shares_price_data_dgk, trading_data_dgk = get_stock_data_from_data_go_kr(kosdaq_codes_datagokr)
    save_data_go_kr_to_csv(total_shares_price_data_dgk, trading_data_dgk)
    !ls # List generated files

